In [1]:
import numpy as np
import pandas as pd
from logistic_dml import DML
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier,RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVR,SVC
from sklearn.linear_model import LogisticRegression, LinearRegression
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, f_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer, PowerTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow import keras 
from scipy import stats
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv( '../../Dataset2_BirthAge20-34_subset10000.csv', low_memory=False)

In [3]:
df[df.birth==0].shape

(5947, 23)

In [4]:
train =df.drop(['Unnamed: 0'],axis=1)

In [5]:
Y = train['birth']
A = train['pill']
X = train.drop(['pill','birth'], axis=1)
K=5

In [6]:
result_gb = DML(
        Y=Y,
        A=A,
        X=X,classifier=GradientBoostingClassifier(), regressor=GradientBoostingRegressor(),random_seed=0, k_folds=K,
    # M_hat_classifier=GradientBoostingClassifier(),
    #          a_hat_classifier=GradientBoostingClassifier()
)      


result_gb.train()
lb_gb, ub_gb, mean_gb, sd_gb = result_gb.significance_testing()

print('beta hat', result_gb.beta_hat)
print('p value', result_gb.p_value)
print('ci', result_gb.ci)
print('lb', lb_gb)
print('ub', ub_gb)

Getting predictions for fold k = 1
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 2
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 3
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting prediction

In [23]:
gbc.best_params_

{'learning_rate': 0.1,
 'max_depth': 3,
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'subsample': 1.0}

In [24]:
gbr.best_params_


{'max_depth': 25, 'min_samples_leaf': 0.01}

In [7]:
rfc_params = {'max_depth': 10,
 'max_features': 'log2',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 100}



In [8]:
rfr_params ={'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 200}

In [9]:
result_rf = DML(
        Y=Y,
        A=A,
        X=X,classifier=RandomForestClassifier(**rfc_params), regressor=RandomForestRegressor(**rfr_params),
    random_seed=0,
    k_folds=K,    

)      
result_rf.train()

lb_rf, ub_rf, mean_rf, sd_rf = result_rf.significance_testing()
print('beta hat', result_rf.beta_hat)
print('p value', result_rf.p_value)
print('ci', result_rf.ci)
print('lb', lb_rf)
print('ub', ub_rf)



Getting predictions for fold k = 1
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 2
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 3
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting prediction

In [15]:
rfc.best_params_

{'max_depth': 100}

In [25]:
param_grid_reg = {
    'C': [0.01,0.1, 1, 10],
    'kernel': ['linear'],
    'epsilon': [0.1, 0.2, 0.5, 1]
}
param_grid_cl = {
    'C': [0.01,0.1, 1, 10],
    'kernel': ['linear'],

}
# Initialize the SVR model
svr = SVR()
svc = SVC()

# Perform Grid Search with neg_mean_squared_error as the scoring metric
grid_search_reg = GridSearchCV(estimator=svr, param_grid=param_grid_reg, cv=5,  verbose=3, scoring='neg_root_mean_squared_error')
grid_search_reg.fit(X, Y)
grid_search_cl = GridSearchCV(estimator=svc, param_grid=param_grid_cl, cv=5, verbose=3, scoring='neg_root_mean_squared_error')
grid_search_cl.fit(X, Y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END C=0.01, epsilon=0.1, kernel=linear;, score=-0.486 total time=   1.2s
[CV 2/5] END C=0.01, epsilon=0.1, kernel=linear;, score=-0.489 total time=   1.1s
[CV 3/5] END C=0.01, epsilon=0.1, kernel=linear;, score=-0.488 total time=   1.2s
[CV 4/5] END C=0.01, epsilon=0.1, kernel=linear;, score=-0.497 total time=   1.2s
[CV 5/5] END C=0.01, epsilon=0.1, kernel=linear;, score=-0.494 total time=   1.2s
[CV 1/5] END C=0.01, epsilon=0.2, kernel=linear;, score=-0.469 total time=   1.1s
[CV 2/5] END C=0.01, epsilon=0.2, kernel=linear;, score=-0.476 total time=   1.1s
[CV 3/5] END C=0.01, epsilon=0.2, kernel=linear;, score=-0.475 total time=   1.1s
[CV 4/5] END C=0.01, epsilon=0.2, kernel=linear;, score=-0.477 total time=   1.1s
[CV 5/5] END C=0.01, epsilon=0.2, kernel=linear;, score=-0.475 total time=   1.1s
[CV 1/5] END C=0.01, epsilon=0.5, kernel=linear;, score=-0.500 total time=   0.0s
[CV 2/5] END C=0.01, epsilon=0.5, ker

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1, 10], 'kernel': ['linear']},
             scoring='neg_root_mean_squared_error', verbose=3)

In [28]:
a=grid_search_cl.best_params_
a['probability']=True

In [29]:


result_sv = DML(
        Y=Y,
        A=A,
        X=X,classifier=SVC(**a), regressor=SVR(**grid_search_reg.best_params_),random_seed=0, k_folds=K,
        # M_hat_classifier=SVC(probability=True,kernel='linear'),
        # a_hat_classifier=SVC(probability=True,kernel='linear'),)      
)

result_sv.train()

lb_sv, ub_sv, mean_sv, sd_sv = result_sv.significance_testing()
print('beta hat', result_sv.beta_hat)
print('p value', result_sv.p_value)
print('ci', result_sv.ci)
print('lb', lb_sv)
print('ub', ub_sv)

Getting predictions for fold k = 1
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 2
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 2
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 3
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 4
Predicting M_hat
Predicting a_hat
Getting predictions for subfold j = 5
Predicting M_hat
Predicting a_hat
Predicting t_hat
Getting predictions for fold k = 3
Predicting M_hat
Getting predictions for subfold j = 1
Predicting M_hat
Predicting a_hat
Getting prediction

In [1]:


model_nnr = keras.Sequential([ 
    keras.layers.Dense(5, input_shape=(X_rs.shape[1],), 
                       activation='sigmoid'),keras.layers.Dense(5, activation='relu'),keras.layers.Dense(1)]) 
model_nnr.compile( 
    optimizer='adam', 
    loss='mean_squared_error', 
    metrics=['accuracy']) 
model_nnc = keras.Sequential([ 
    keras.layers.Dense(5, input_shape=(X_rs.shape[1],), 
                       activation='relu'),keras.layers.Dense(5, activation='relu'),keras.layers.Dense(units=1, activation='sigmoid')
]) 
model_nnc.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 
result_nn = DML(
        Y=y_rs,
        A=A_rs,
        X=X_rs,classifier=model_nnc , regressor=model_nnr,random_seed=0, k_folds=K)      


result_nn.train()

lb_nn, ub_nn, mean_nn, sd_nn = result_sv.significance_testing()
print('beta hat', result_nn.beta_hat)
print('p value', result_nn.p_value)
print('ci', result_nn.ci)
print('lb', lb_nn)
print('ub', ub_nn)

NameError: name 'keras' is not defined

for tensorflow<=2.5, predict_proba is  deprectaed and no longer in use

In [30]:
data = {'ML':['GB','RF','SVM'],'Beta':[result_gb.beta_hat,result_rf.beta_hat,result_sv.beta_hat],
       'CI_LB':[result_gb.ci[0],result_rf.ci[0],result_sv.ci[0]], 
        "CI_UB":[result_gb.ci[1],result_rf.ci[1],result_sv.ci[1]],
        "p_val":[result_gb.p_value,result_rf.p_value,result_sv.p_value]}
                        

In [31]:
data=pd.DataFrame(data)
data

,ML,Beta,CI_LB,CI_UB,p_val
0,GB,-0.154330,-0.667724,0.359064,0.725
1,RF,-0.163758,-0.274508,-0.053007,0.165
2,SVM,-0.172758,-0.344280,-0.001235,0.325
